<a href="https://colab.research.google.com/github/LucasMirandaVS/estudos_python/blob/main/Cleaning_Ace_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Step 2: Export the raw data to a CSV file for traceability and downstream processing
csv_path = "May2025RawData.csv"
df_raw.to_csv(csv_path, index=False)

In [ ]:
# Step 3: Load the CSV file just created (acts as a checkpoint)
try:
    df = pd.read_csv(csv_path, sep=",", encoding="utf-8")
except pd.errors.ParserError:
    df = pd.read_csv(csv_path, sep=";", encoding="utf-8", error_bad_lines=False)

/tmp/ipython-input-3638582853.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path, sep=",", encoding="utf-8")


In [ ]:
# Step 4: Create a unique key for each tariff line by combining Entry Summary Number and Entry Summary Line Number
# This ensures that we treat each line as a unique transaction for aggregation purposes
df["Entry Summary Code"] = (
    df["Entry Summary Number"].astype(str) + "-" + df["Entry Summary Line Number"].astype(str)
)

In [ ]:
# Step 5: Group by the unique line (Entry Summary Code) and HTS Number to prevent double-counting tariffs
# We sum the values within each group to consolidate any internal duplications
grouped = (
    df.groupby(["Entry Summary Code", "HTS Number - Full"], dropna=False)
    .agg({
        "Line Tariff Duty Amount": "sum",
        "Antidumping Duty Amount": "sum",
        "Countervailing Duty Amount": "sum"
    })
    .reset_index()
)

In [ ]:
# Step 6: Now group by HTS Number only, aggregating across all unique entry lines
# This gives us the final total tariffs per HTS code, with duplications removed
final = (
    grouped.groupby("HTS Number - Full", dropna=False)[
        ["Line Tariff Duty Amount", "Antidumping Duty Amount", "Countervailing Duty Amount"]
    ]
    .sum()
    .reset_index()
)


In [ ]:
# Step 7: Rename columns for clarity in the output file
final.columns = [
    "HTS Number",
    "Total Line Tariff Duty Amount",
    "Total Antidumping Duty Amount",
    "Total Countervailing Duty Amount"
]

In [ ]:
# Step 8: Export the cleaned and aggregated data to a new CSV file
final.to_csv("corrected_tariffs_by_hts.csv", index=False)